In [1]:
import numpy as np
import keras, cv2, random, os, csv, json
from pathlib import Path
from os import listdir
from skimage.io import imread
from skimage.transform import resize
from keras.models import Model, Sequential
from keras.preprocessing.image import img_to_array, load_img
from keras import layers
from keras.layers import GlobalAveragePooling2D, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, Dropout, Flatten
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import MaxPooling3D
from keras.layers import AveragePooling3D
from keras.layers import GlobalAveragePooling3D
import time
from keras.utils import layer_utils
import os.path
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
def partition_list_csv(num_classes):
    csv_dir = "/home/t-ziwang/action_classification/five-video-classification-methods/data/data_file.csv"
    with open(csv_dir, 'r') as f:
        reader = csv.reader(f)
        csv_list = np.array(list(reader))
    partition = {}
    label = {}
    t_num = 0
    v_num = 0
    class_no = 0
    class_list = np.unique(csv_list[:,1])
    np.random.shuffle(class_list)
    for iclass in class_list:
        if class_no < num_classes:
            label[iclass] = class_no
            class_no = class_no + 1
    for line in range(np.size(csv_list,0)):
        if csv_list[line,1] in label.keys():
            if csv_list[line,0] == "train":
                if random.random() <= 0.8:
                    # train
                    if t_num == 0:
                        t_list = csv_list[line,-3:]
                        t_num = t_num + 1
                    else:
                        t_list = np.vstack((t_list, csv_list[line,-3:]))
                else:
                    # valid
                    if v_num == 0:
                        v_list = csv_list[line,-3:]
                        v_num = v_num + 1
                    else:
                        v_list = np.vstack((v_list, csv_list[line,-3:]))
    partition['train'] = t_list
    partition['validation'] = v_list
    return partition, label

def partition_per_frame(partlist, clip_length = 15):
    for i, item in enumerate(partlist):
        if i == 0:
            out_list = np.array([[item[0], item[1], idx + 1] for idx in range(int(item[2]) - clip_length - 1)])
        else:
            out_list = np.concatenate((out_list, np.array([[item[0], item[1], idx + 1] for idx in range(int(item[2]) - clip_length - 1)])), axis = 0)
            
    return out_list

In [3]:
class precomp():
    def __init__(self, trs_res):
        self.row, self.col = trs_res
        
        dn = 1/float(np.min((self.col, self.row)))
        log_rho_axis = np.linspace(np.log(dn), 0, self.col)
        theta_axis = np.linspace(0, np.pi*(1-dn), self.row)
        x0 = self.col/2.0
        y0 = self.row/2.0
        rmax = np.sqrt(x0**2 + y0**2)
        lp_table_x = np.empty((self.row, self.col))
        lp_table_y = np.empty((self.row, self.col))
        self.mask = np.ones((self.row, self.col))
        for ii in range(self.row):
            for jj in range(self.col):
                rho = np.exp(log_rho_axis[jj])
                theta = theta_axis[-ii-1]
                xx = x0 + rho * np.cos(theta) * rmax
                yy = y0 - rho * np.sin(theta) * rmax
                if xx < 0 or xx > (self.col - 1) or yy < 0 or yy > (self.row - 1):
                    self.mask[ii,jj] = 0
                lp_table_x[ii,jj] = xx
                lp_table_y[ii,jj] = yy
        lp_table_x = lp_table_x.flatten()
        lp_table_y = lp_table_y.flatten()
        cleft = np.floor(lp_table_x).astype(int)
        cright = cleft + 1
        ax = lp_table_x - cleft
        cup = np.floor(lp_table_y).astype(int)
        cdown = cup + 1
        ay = lp_table_y - cup
        self.iul = cup*self.col + cleft
        self.iur = cup*self.col + cright
        self.idl = cdown*self.col + cleft
        self.idr = cdown*self.col + cright
        self.aul = np.multiply((1-ax), (1-ay))
        self.aur = np.multiply(ax, (1-ay))
        self.adl = np.multiply((1-ax), ay)
        self.adr = np.multiply(ax, ay)
        self.mask = self.mask.flatten()

In [4]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, 
                 work_dir="/home/t-ziwang/action_classification/five-video-classification-methods/data/train/", 
                 batch_size=32, dim=(224,224), n_channels=1, op_res = (256, 256), skip = 1, ms_skip = None,
                 n_classes=10, mode = 'vid', trs_res = (256, 256), trs_precomp = None,
                 psf = None, stack = False, shuffle=True, verbose = 0):
        'Initialization'
        self.work_dir = work_dir
        self.dim = dim
        self.dh, self.dw = self.dim
        self.h1 = self.dh // 2
        self.w1 = self.dw // 2
        self.h2 = self.dh - self.h1
        self.w2 = self.dw - self.h2
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.mode = mode
        self.n_channels = n_channels
        self.skip = skip
        if self.mode == "vid-trs" or self.mode == "ca-trs" or self.mode == "ms-trs":
            if self.mode == "ms-trs":
                self.row, self.col = trs_res
                self.n_channels = 13
                self.ms_skip = ms_skip
                self.frame_idx = self.get_frame_idx()
                self.dim3 = self.get_dim3()
            else:
                self.n_channels = self.n_channels + 1
                self.row, self.col = trs_res
            if trs_precomp == None:
                self.trs_precomp_flag = 0
                self.precompute_lp = 0
            else:
                self.trs_precomp_flag = 1
                self.precompute_lp = 1
                self.trs_precomp = trs_precomp
        if self.mode == "ca" or self.mode == "ca-trs":
            self.psf = psf
            self.psf_res = (np.size(self.psf, 0), np.size(self.psf, 1))
            self.psf_fft = np.fft.fft2(self.psf)
        self.op_res = op_res
        self.n_classes = n_classes
        
        self.stack = stack
        self.shuffle = shuffle
        self.verbose = verbose
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(0.3*np.size(self.list_IDs, 0) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k,:] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(np.size(self.list_IDs,0))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def read_image_sequence(self, vid_dir, img_basename, start_idx):
        vid = [img_to_array(load_img(os.path.join(vid_dir, img_basename + '%0*d' % (4, start_idx + self.skip * iframe) + ".jpg"), target_size = self.op_res, grayscale = True)) for iframe in range(self.n_channels)]
        return np.squeeze((np.array(vid) / 255.).astype(np.float32))
    
    def get_frame_idx(self):
        return np.array([np.arange(0, self.n_channels, iskip) for iskip in self.ms_skip])
    
    def get_dim3(self):
        dim3 = 0
        for i in self.ms_skip:
            dim3 = dim3 + len(np.arange(0, self.n_channels, i)) - 1
        return dim3*2
            
    def norm_max(self, img):
        return img/np.amax(img)
    
    def crop_video(self, vid):
        H, W = self.op_res
        h = random.randint(0, H - self.dh - 1)
        w = random.randint(0, W - self.dw - 1)
        return vid[:, h:(h + self.dh), w:(w + self.dw)]
    
    def image_downsize(self, img):
        return resize(img, self.dim, mode = 'reflect', anti_aliasing=True)
        
    def video2ca(self, vid):
        return np.array([self.image_downsize(np.absolute(np.fft.ifft2(np.multiply(np.fft.fft2(resize(vid[i,:,:], self.psf_res, mode = 'reflect', anti_aliasing=True)), self.psf_fft)))) for i in range(np.size(vid,0))])
    
    def crop_resize_video(self, vid):
        H, W = self.op_res
        h = random.randint(0, H - self.dh - 1)
        w = random.randint(0, W - self.dw - 1)
        return np.array([resize(vid[i, h:(h+self.dh), w:(w+self.dw)], (self.row, self.col), mode = 'reflect', anti_aliasing=True) for i in range(np.size(vid, 0))])
    
    def crop_center_img(self, img):
        Hc = self.row // 2
        Wc = self.col // 2
        return img[(Hc - self.h1): (Hc + self.h2), (Wc - self.w1): (Wc + self.w2)]
    
    def cropy_resizex_img(self, img):
        Hc = self.row // 2
        Wc = self.col // 2
        return resize(img[(Hc - self.h1): (Hc + self.h2), :] , (self.dh, self.dw), mode = 'reflect', anti_aliasing = True)
    
    def phase_corr(self,img1,img2):
        self.img1_fft_t = np.fft.fftshift(np.fft.fft2(img1))
        self.img2_fft_t = np.fft.fftshift(np.fft.fft2(img2))
        temp_dot = np.multiply(self.img1_fft_t, np.conj(self.img2_fft_t))
        cps = temp_dot/(np.absolute(temp_dot)+np.finfo(float).eps)
        cps_ifft = np.nan_to_num(np.absolute(np.fft.fftshift(np.fft.ifft2(cps))))
        return cps_ifft
    
    def log_polar_precomp(self, img):
        img_flat = img.flatten()
        if self.precompute_lp == 0:
            dn = 1/float(self.col)
            log_rho_axis = np.linspace(np.log(dn), 0, self.col)
            theta_axis = np.linspace(0, np.pi*(1-dn), self.row)
            x0 = self.col/2.0
            y0 = self.row/2.0
            rmax = np.sqrt(x0**2 + y0**2)
            lp_table_x = np.empty((self.row, self.col))
            lp_table_y = np.empty((self.row, self.col))
            self.mask = np.ones((self.row, self.col))
            for ii in range(self.row):
                for jj in range(self.col):
                    rho = np.exp(log_rho_axis[jj])
                    theta = theta_axis[-ii-1]
                    xx = x0 + rho * np.cos(theta) * rmax
                    yy = y0 - rho * np.sin(theta) * rmax
                    if xx < 0 or xx > (self.col - 1) or yy < 0 or yy > (self.row - 1):
                        self.mask[ii,jj] = 0
                    lp_table_x[ii,jj] = xx
                    lp_table_y[ii,jj] = yy
            lp_table_x = lp_table_x.flatten()
            lp_table_y = lp_table_y.flatten()
            cleft = np.floor(lp_table_x).astype(int)
            cright = cleft + 1
            ax = lp_table_x - cleft
            cup = np.floor(lp_table_y).astype(int)
            cdown = cup + 1
            ay = lp_table_y - cup
            self.iul = cup*self.col + cleft
            self.iur = cup*self.col + cright
            self.idl = cdown*self.col + cleft
            self.idr = cdown*self.col + cright
            self.aul = np.multiply((1-ax), (1-ay))
            self.aur = np.multiply(ax, (1-ay))
            self.adl = np.multiply((1-ax), ay)
            self.adr = np.multiply(ax, ay)
            self.mask = self.mask.flatten()
            self.precompute_lp = 1
        if self.trs_precomp_flag == 0:
            canvas = np.multiply(self.mask, np.multiply(img_flat[self.iul], self.aul) + np.multiply(img_flat[self.iur], self.aur) + np.multiply(img_flat[self.idl], self.adl) + np.multiply(img_flat[self.idr], self.adr))    
        else:
            canvas = np.multiply(self.trs_precomp.mask, np.multiply(img_flat[self.trs_precomp.iul], self.trs_precomp.aul) + np.multiply(img_flat[self.trs_precomp.iur], self.trs_precomp.aur) + np.multiply(img_flat[self.trs_precomp.idl], self.trs_precomp.adl) + np.multiply(img_flat[self.trs_precomp.idr], self.trs_precomp.adr))    
        return canvas.reshape(self.row, self.col)
    
    def TRSmaps(self,img1,img2):
        tmap = self.phase_corr(img1,img2)
        img1_lp = self.log_polar_precomp(np.absolute(self.img1_fft_t))
        img2_lp = self.log_polar_precomp(np.absolute(self.img2_fft_t))
        rsmap = self.phase_corr(img1_lp,img2_lp)
        return tmap, rsmap
    
    def trs_crop(self, img1, img2):
        tmap, rsmap = self.TRSmaps(img1, img2)
        return np.concatenate((np.expand_dims(self.crop_center_img(tmap), axis = 0), np.expand_dims(self.cropy_resizex_img(rsmap), axis = 0)), axis = 0)
    
    def ms_trs(self, vid, scale):
        [[self.trs_crop] for iscale in scales]
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        y = np.empty((self.batch_size), dtype=int)
        if self.mode == 'vid' or self.mode == 'ca':
            X = np.empty((self.batch_size, *self.dim, self.n_channels))
        elif self.mode == 'ms-trs':
            X = np.empty((self.batch_size, *self.dim, self.dim3))
        elif self.stack == True:
            X = np.empty((self.batch_size, *self.dim, (self.n_channels-1)*2))
        else:
            X = np.empty((self.batch_size, 2, *self.dim, (self.n_channels-1)))
        # Generate data
        time_prep_samp = time.time()
        if self.verbose == 1:
            print("Preparing samples...", self.mode)
        for i, ID in enumerate(list_IDs_temp):
            if self.verbose == 1:
                print("i", i, "ID", ID)
            if self.mode == 'vid':
                # Store sample
#                 rand_start_idx = random.randint(1, int(ID[2]) - self.n_channels * self.skip - 1)
#                 time_read = time.time()
#                 vid = self.read_image_sequence(os.path.join(self.work_dir, ID[0]),ID[1] + "-", rand_start_idx)
#                 time_crop_trans = time.time()
                vid = self.read_image_sequence(os.path.join(self.work_dir, ID[0]),ID[1] + "-", int(ID[2]))
                X[i,] = np.transpose(self.crop_video(vid), (1,2,0))
                # Store class
                y[i] = self.labels[ID[0]]
            elif self.mode == 'vid-trs':
#                 rand_start_idx = random.randint(1, int(ID[2]) - self.n_channels * self.skip - 1)
#                 vid = self.crop_resize_video(self.read_image_sequence(os.path.join(self.work_dir, ID[0]),ID[1] + "-", rand_start_idx))
                vid = self.crop_resize_video(self.read_image_sequence(os.path.join(self.work_dir, ID[0]),ID[1] + "-", int(ID[2])))

                if self.stack == True:
                    X[i,] = np.transpose(np.resize(np.array([self.trs_crop(vid[j,:,:], vid[j+1,:,:]) for j in range(self.n_channels - 1)]), ((self.n_channels-1)*2, *self.dim)), (1,2,0))
                else:
                    X[i,] = np.transpose(np.array([self.trs_crop(vid[j,:,:], vid[j+1,:,:]) for j in range(self.n_channels - 1)]), (1,2,3,0))
                y[i] = self.labels[ID[0]]
            elif self.mode == 'ms-trs':
#                 rand_start_idx = random.randint(1, int(ID[2]) - self.n_channels - 1)
#                 vid = self.crop_resize_video(self.read_image_sequence(os.path.join(self.work_dir, ID[0]),ID[1] + "-", rand_start_idx))
                vid = self.crop_resize_video(self.read_image_sequence(os.path.join(self.work_dir, ID[0]),ID[1] + "-", int(ID[2])))
                for iskip in range(len(self.ms_skip)):
                    temp = np.transpose(np.reshape(np.array([self.trs_crop(vid[self.frame_idx[iskip][j],:,:], vid[self.frame_idx[iskip][j+1],:,:]) 
                                                             for j in range(len(self.frame_idx[iskip]) - 1)]), (2*(len(self.frame_idx[iskip])-1),*self.dim)), (1,2,0))
                    if iskip == 0:
                        ms_trs = temp
                    else:
                        ms_trs = np.concatenate((ms_trs, temp), axis = 2)
                X[i,] = ms_trs
                y[i] = self.labels[ID[0]]
#                     print(temp.shape)
#                     plt.figure()
#                     plt.subplot(121)
#                     plt.imshow(temp[:,:,1]**0.1)
#                     plt.subplot(122)
#                     plt.imshow(temp[:,:,2]**0.1)
#                 break
#                 
#                 
#                 np.transpose(np.resize(np.array([self.trs_crop(vid[j,:,:], vid[j+1,:,:]) for j in range(self.n_channels - 1)]), ((self.n_channels-1)*2, *self.dim)), (1,2,0))
            elif self.mode == 'ca':
                rand_start_idx = random.randint(1, int(ID[2]) - self.n_channels * self.skip - 1)
                vid = self.read_image_sequence(os.path.join(self.work_dir, ID[0]),ID[1] + "-", rand_start_idx)
#                 plt.figure()
#                 plt.imshow(vid[0,:,:])
#                 temp = np.transpose(self.video2ca(self.crop_video(vid)), (1,2,0))
                
#                 plt.figure()
#                 plt.imshow(temp[:,:,0])
                X[i,] = np.transpose(self.video2ca(self.crop_video(vid)), (1,2,0))
                y[i] = self.labels[ID[0]]
        if self.verbose == 1:
            print("Finished preparing samples! size:", np.array(X).shape, "time taken:", time.time() - time_prep_samp)
        return np.array(X), keras.utils.to_categorical(y, num_classes=self.n_classes)

def norm_sum(img):
        return img/np.sum(img)

In [5]:
def conv_2d(num_classes, input_shape):
    model = Sequential()
    
    model.add(Conv2D(64, (3, 3), strides = (1, 1), padding = "same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Conv2D(64, (3, 3), strides = (1, 1), padding = "same"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
    
    for i in range(2):
        model.add(Conv2D(128, (3, 3), strides = (1, 1), padding = "same"))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
    
    for i in range(3):
        model.add(Conv2D(256, (3, 3), strides = (1, 1), padding = "same"))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
    
    for i in range(3):
        model.add(Conv2D(512, (3, 3), strides = (1, 1), padding = "same"))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
       
    for i in range(3):
        model.add(Conv2D(512, (3, 3), strides = (1, 1), padding = "same"))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
    
    model.add(Flatten())   
    
    for i in range(2):
        model.add(Dense(4096, activation='relu'))
        model.add(Dropout(0.5))

    model.add(Dense(num_classes, activation='softmax'))
     
    return model

In [6]:
num_classes = 5
# partition, labels = partition_list_csv(num_classes)
save_dir = "/home/t-ziwang/action_classification/winston_action_classification/"
# np.save(save_dir + "ucf-partition-labels-" + "%03dc.npy" % num_classes, (partition, labels))
partition, labels = np.load(save_dir + "ucf-partition-labels-" + "%03dc.npy" % num_classes)
print("training video #", np.size(partition['train'], 0))
print("validation video #", np.size(partition['validation'], 0))
print(labels)

training video # 363
validation video # 88
{'Fencing': 0, 'Shotput': 1, 'BabyCrawling': 2, 'BaseballPitch': 3, 'TaiChi': 4}


In [7]:
partition['train'] = partition_per_frame(partition['train'], clip_length = 15)
partition['validation'] = partition_per_frame(partition['validation'], clip_length = 15)
print("training video #", np.size(partition['train'], 0))
print(partition['train'])
print("validation video #", np.size(partition['validation'], 0))
print(labels)

training video # 41841
[['Shotput' 'v_Shotput_g16_c05' '1']
 ['Shotput' 'v_Shotput_g16_c05' '2']
 ['Shotput' 'v_Shotput_g16_c05' '3']
 ...
 ['BabyCrawling' 'v_BabyCrawling_g16_c06' '174']
 ['BabyCrawling' 'v_BabyCrawling_g16_c06' '175']
 ['BabyCrawling' 'v_BabyCrawling_g16_c06' '176']]
validation video # 9759
{'Fencing': 0, 'Shotput': 1, 'BabyCrawling': 2, 'BaseballPitch': 3, 'TaiChi': 4}


In [8]:
trs_res = (512, 512)
trs_precomp = precomp(trs_res)
#psf = norm_sum(imread("/home/t-ziwang/action_classification/MLS_psf/mask-MLS-256-1x-0s-1r-(0,0)-1.0-synth.png", as_gray = True))

In [9]:
num_channels = 13
train_bsz = 32
valid_bsz = 32
# Parameters
train_params = {'work_dir': "/home/t-ziwang/action_classification/five-video-classification-methods/data/train/",
          'dim': (224,224),
          'batch_size': train_bsz,
          'n_classes': num_classes,
          'n_channels': num_channels,
          'op_res': (240, 320), # operating resolution
          'trs_res': trs_res, # resolution for computing trs
          'trs_precomp': trs_precomp, # precomputed trs log-polar class object
          'skip': 1, # skip frames, 1 -- no skip
          'ms_skip': [2,3,4,6], # skip frames, 1 -- no skip
          'mode': 'vid', # "vid", "vid-trs", "ca", "ca-trs"
          'psf': None, 
          'stack': True, # for trs generation, if true, output stacked trs, otherwise output two-stream tuple
          'shuffle': True,
          'verbose': 0}

val_params = train_params.copy()
val_params['batch_size'] = valid_bsz
# Generators
# training_generator = DataGenerator(partition['train'], labels, **train_params)
# t_mstrs = time.time()
# X, y = training_generator.__getitem__(0)
# print(X.shape)
# print(time.time() - t_mstrs)

In [ ]:
epochs = 10
verbose = 1
mode_list = ['ms-trs']
history = {}
for imode in mode_list:
    train_params['mode'] = imode
    val_params['mode'] = imode
    if imode == 'vid' or imode == 'ca':
        train_params['n_channels'] = num_channels
        val_params['n_channels'] = num_channels
        model = conv_2d(train_params['n_classes'], (*train_params['dim'], train_params['n_channels']))
    elif imode == 'ms-trs':
        model = conv_2d(train_params['n_classes'], (*train_params['dim'], 30))
    else:
        train_params['n_channels'] = num_channels - 1
        val_params['n_channels'] = num_channels - 1
        model = conv_2d(train_params['n_classes'], (*train_params['dim'], train_params['n_channels']*2))
    training_generator = DataGenerator(partition['train'], labels, **train_params)
    validation_generator = DataGenerator(partition['validation'], labels, **val_params)
    adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

    history[imode] = model.fit_generator(generator=training_generator,
                        validation_data=validation_generator,
                        use_multiprocessing=True, epochs=epochs, verbose = verbose,
                        workers=6, initial_epoch = 0)
    model.save(save_dir + train_params['mode'] + "-%03dcl" % num_classes + "-%02dch" % num_channels + "-%03dtrsres" % trs_res[0] + "-%03dep-model.h5" % epochs)
    history_dict = history[imode].history
    json.dump(history_dict, open(save_dir + train_params['mode'] + "-%03dcl" % num_classes + "-%02dch" % num_channels + "-%03dtrsres" % trs_res[0] + "-%03dep-history.json" % epochs, 'w'))

Epoch 1/10
392/392 [==============================] - 27035s 69s/step - loss: 1.7886 - acc: 0.4281 - val_loss: 1.1758 - val_acc: 0.5532
Epoch 2/10
392/392 [==============================] - 26226s 67s/step - loss: 1.0854 - acc: 0.5977 - val_loss: 2.8672 - val_acc: 0.3080
Epoch 3/10
392/392 [==============================] - 25022s 64s/step - loss: 0.9472 - acc: 0.6544 - val_loss: 1.9949 - val_acc: 0.2229
Epoch 4/10
137/392 [=========>....................] - ETA: 3:36:44 - loss: 0.8529 - acc: 0.6880

In [ ]:
model.save(save_dir + train_params['mode'] + "-%03dcl" % num_classes + "-%02dch" % num_channels + "-%03dtrsres" % trs_res[0] + "-%03dep-model.h5" % epochs)